### Pure State Simulation
Cirq comes with built-in Python simulators for testing small circuits. They two main types of simulations that Cirq supports are pure state and mixed state. The pure state simulators are supported by Cirq.simulators and mixed state simulators with DensityMatrixSimulators

In [1]:
import cirq

q0 = cirq.GridQubit(0, 0)
q1 = cirq.GridQubit(1, 0)

def basic_circuit(meas=True):
    sqrt_x = cirq.X**0.5
    yield sqrt_x(q0), sqrt_x(q1)
    yield cirq.CZ(q0, q1)
    yield sqrt_x(q0), sqrt_x(q1)
    if meas:
        yield cirq.measure(q0, key='q0'), cirq.measure(q1, key='q1')

circuit = cirq.Circuit()
circuit.append(basic_circuit())

print(circuit)

(0, 0): ───X^0.5───@───X^0.5───M('q0')───
                   │
(1, 0): ───X^0.5───@───X^0.5───M('q1')───


We can simulate this by creating a cirq.Simulator and passing the circuit into its run method

In [2]:
from cirq import Simulator
simulator = Simulator()
result = simulator.run(circuit)

print(result)

q0=1
q1=0


### Qubit and Amplitude Ordering
The simplest qubit_order value you can provide is a list of the qubits in the desired ordered. Any qubits from the circuit that are not in the list will be ordered using the default __str__ ordering, but come after qubits that are in the list. Be aware that all qubits in the list are included in the simulation, even if they are not operated on by the circuit.

The mapping from the order of the qubits to the order of the amplitudes in the wave function can be tricky to understand. Basically, it is the same as the ordering used by numpy.kron:

In [5]:
import numpy as np
circuit = cirq.Circuit()
circuit.append(basic_circuit(False))
result = simulator.simulate(circuit, qubit_order=[q0, q1])

print(np.around(result.final_state, 3))
outside = [1, 10]
inside = [1, 2]
print(np.kron(outside, inside))

[0.5+0.j  0. +0.5j 0. +0.5j 0.5+0.j ]
[ 1  2 10 20]


In [6]:
## The first qubit’s computational basis values are looped over in the outermost loop, the last qubit’s computational basis values are looped over in the inner-most loop, etc.:
i = 0
for first in [0, 1]:
    for second in [0, 1]:
        print('amps[{}] is for first={}, second={}'.format(i, first, second))
        i += 1

amps[0] is for first=0, second=0
amps[1] is for first=0, second=1
amps[2] is for first=1, second=0
amps[3] is for first=1, second=1


###  Stepping through a Circuit
When debugging, it is useful to not just see the end result of a circuit, but to inspect, or even modify, the state of the system at different steps in the circuit.

To support this, Cirq provides a method to return an iterator over a Moment by Moment simulation. This method is named simulate_moment_steps:

In [7]:
circuit = cirq.Circuit()
circuit.append(basic_circuit())
for i, step in enumerate(simulator.simulate_moment_steps(circuit)):
    print('state at step %d: %s' % (i, np.around(step.state_vector(), 3)))

state at step 0: [0. +0.5j 0.5+0.j  0.5+0.j  0. -0.5j]
state at step 1: [0. +0.5j 0.5+0.j  0.5+0.j  0. +0.5j]
state at step 2: [0.5+0.j  0. +0.5j 0. +0.5j 0.5+0.j ]
state at step 3: [0.+0.j 0.+0.j 0.+0.j 1.+0.j]


### Monte Carlo Simulations of Noise
Some noise models can be thought of as randomly applying unitary evolutions with different probabilities. Such noise models are amenable to Monte Carlo simulation.

An example of such a noise model is the bit flip channel. This channel randomly applied either does nothing (identity) or applies a Pauli cirq.X gate:

In [8]:
q = cirq.NamedQubit('a')
circuit = cirq.Circuit(cirq.bit_flip(p=0.2)(q), cirq.measure(q))
simulator = cirq.Simulator()
result = simulator.run(circuit, repetitions=100)
print(result.histogram(key='a'))

Counter({0: 74, 1: 26})


### Parameterized Values and Studies
In addition to circuit gates with fixed values, Cirq also supports gates which can have Symbol value (see Gates). These are values that can be resolved at run-time.

For simulators, these values are resolved by providing a ParamResolver. A ParamResolver provides a map from the Symbol’s name to its assigned value.

In [9]:
import sympy
rot_w_gate = cirq.X**sympy.Symbol('x')
circuit = cirq.Circuit()
circuit.append([rot_w_gate(q0), rot_w_gate(q1)])
for y in range(5):
    resolver = cirq.ParamResolver({'x': y / 4.0})
    result = simulator.simulate(circuit, resolver)
    print(np.round(result.final_state, 2))

[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
[ 0.6 +0.6j   0.25-0.25j  0.25-0.25j -0.1 -0.1j ]
[0. +0.5j 0.5+0.j  0.5+0.j  0. -0.5j]
[-0.1 +0.1j   0.25+0.25j  0.25+0.25j  0.6 -0.6j ]
[0.+0.j 0.+0.j 0.+0.j 1.+0.j]
